In [1]:
import pandas as pd
from functools import reduce

In [44]:
emp_personal_info_df = pd.read_csv("Employee_Information.csv")
emp_travel_info_df = pd.read_csv("Employee_Travel_Information.csv")
individually_travelled_emp_file_df = pd.read_csv("Individually_Travelled_Employee_File.csv")
travel_agency_file_df = pd.read_csv("Travel_Agency_File.csv")
emissions_lookup_for_car_df = pd.read_csv("Emission_Details_lookup_table_for_Car.csv")
emissions_lookup_road_travel_other_than_cars_df = pd.read_csv("Emission_Details_lookup_table_other_than_cars.csv")
emissions_lookup_rail_air_df = pd.read_csv("Emission_Details_lookup_table_rail_air.csv")
emissions_lookup_hotel_df = pd.read_csv("Emission_Details_lookup_table_hotel.csv")

category_name = "Business Travel"
scope = 3
emp_travel_info_df['NoOfEmployeesInGroup'] = emp_travel_info_df['NoOfEmployeesInGroup'].fillna(1)
emp_travel_info_df = emp_travel_info_df.fillna('Not Admissible')

individually_travelled_emp_file_df['AmountSpentOnFuelIn$'] = individually_travelled_emp_file_df['AmountSpentOnFuelIn$'].fillna(0)
individually_travelled_emp_file_df['Fuel Consumed'] = individually_travelled_emp_file_df['Fuel Consumed'].fillna(0)
individually_travelled_emp_file_df['ElectricityConsumed'] = individually_travelled_emp_file_df['ElectricityConsumed'].fillna(0)
individually_travelled_emp_file_df['Employees per Vehicle'] = individually_travelled_emp_file_df['Employees per Vehicle'].fillna(1)
individually_travelled_emp_file_df['EngineCapacity'] = individually_travelled_emp_file_df['EngineCapacity'].fillna(0)
individually_travelled_emp_file_df['Fuel price per L'] = individually_travelled_emp_file_df['Fuel price per L'].fillna(0)
individually_travelled_emp_file_df['AmountSpentOnTravelModeIn$'] = individually_travelled_emp_file_df['AmountSpentOnTravelModeIn$'].fillna(0)
individually_travelled_emp_file_df['DistanceTravelledInTheVehicle'] = individually_travelled_emp_file_df['DistanceTravelledInTheVehicle'].fillna(0)
individually_travelled_emp_file_df['TotalNightsStayed'] = individually_travelled_emp_file_df['TotalNightsStayed'].fillna(0)
individually_travelled_emp_file_df['EmployeeID'] = individually_travelled_emp_file_df['EmployeeID'].astype(str)
individually_travelled_emp_file_df = individually_travelled_emp_file_df.fillna('Not Admissible')

travel_agency_file_df['AmountSpentOnFuelIn$'] = travel_agency_file_df['AmountSpentOnFuelIn$'].fillna(0)
travel_agency_file_df['Fuel Consumed'] = travel_agency_file_df['Fuel Consumed'].fillna(0)
travel_agency_file_df['ElectricityConsumed'] = travel_agency_file_df['ElectricityConsumed'].fillna(0)
travel_agency_file_df['Employees per Vehicle'] = travel_agency_file_df['Employees per Vehicle'].fillna(1)
travel_agency_file_df['EngineCapacity'] = travel_agency_file_df['EngineCapacity'].fillna(0)
travel_agency_file_df['Fuel price per L'] = travel_agency_file_df['Fuel price per L'].fillna(0)
travel_agency_file_df['AmountSpentOnTravelModeIn$'] = travel_agency_file_df['AmountSpentOnTravelModeIn$'].fillna(0)
travel_agency_file_df['DistanceTravelledInTheVehicle'] = travel_agency_file_df['DistanceTravelledInTheVehicle'].fillna(0)
travel_agency_file_df['TotalNightsStayed'] = travel_agency_file_df['TotalNightsStayed'].fillna(0)
travel_agency_file_df = travel_agency_file_df.fillna('Not Admissible')

emissions_lookup_for_car_df['EngineCapacity'] = emissions_lookup_for_car_df['EngineCapacity'].fillna(0)
emissions_lookup_for_car_df['Ef for fuels kgCO2e/L'] = emissions_lookup_for_car_df['Ef for fuels kgCO2e/L'].fillna(0)
emissions_lookup_for_car_df['consumed L per 100km'] = emissions_lookup_for_car_df['consumed L per 100km'].fillna(0)

emissions_lookup_road_travel_other_than_cars_df['Ef for fuels kgCO2e/L'] = emissions_lookup_road_travel_other_than_cars_df['Ef for fuels kgCO2e/L'].fillna(0)

emissions_lookup_rail_air_df['VehicleSubCategory'] = emissions_lookup_rail_air_df['VehicleSubCategory'].fillna('Not Admissible')
emissions_lookup_rail_air_df['VersionOrClass'] = emissions_lookup_rail_air_df['VersionOrClass'].fillna('Not Admissible')
emissions_lookup_rail_air_df['Ef for fuels kgCO2e/L'] = emissions_lookup_rail_air_df['Ef for fuels kgCO2e/L'].fillna(0)

input_dfs = [emp_personal_info_df, emp_travel_info_df, travel_agency_file_df, individually_travelled_emp_file_df]
combined_input_dfs = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                                              left_index=False, right_index=False, 
                                              how='outer'), input_dfs)

combined_input_dfs = combined_input_dfs.sort_values(by=['EmployeeID','OrganisationName','WorkRegion'])
combined_input_dfs['EngineCapacity'].astype('float')

combined_input_with_emission_df = pd.merge(combined_input_dfs, emissions_lookup_for_car_df, on=['VehicleManufacturer','VehicleSubCategory','EngineCapacity','VersionOrClass'] , how = 'left')
final_combined_input_with_emission_df = combined_input_with_emission_df[['EmployeeName','EmployeeID','WorkRegion','OrganisationName','Role','TravelledCountry','TravelledDate','WentWithGroup','NoOfEmployeesInGroup','Employees per Vehicle','TrasportMode','TotalNightsStayed','Ef for fuels kgCO2e/L','AmountSpentOnTravelModeIn$','VehicleManufacturer','VehicleSubCategory','EngineCapacity','VersionOrClass','FuelType','DistanceTravelledInTheVehicle','Fuel price per L','AmountSpentOnFuelIn$','Fuel Consumed','ElectricityConsumed','consumed L per 100km','AgencyName','Emission(g/km)','EEIO ef']].copy()
final_combined_input_with_emission_df.insert(22,"Emission(kg/km)", True)
final_combined_input_with_emission_df['Emission(kg/km)'] = final_combined_input_with_emission_df['Emission(g/km)'] / 1000

final_emission_df = pd.merge(final_combined_input_with_emission_df, emissions_lookup_road_travel_other_than_cars_df, on=['TravelledCountry','TrasportMode'] , how = 'left')

final_emission_df['Ef for fuels kgCO2e/L_x'] = final_emission_df['Ef for fuels kgCO2e/L_x'].fillna(0)
final_emission_df['Ef for fuels kgCO2e/L_y'] = final_emission_df['Ef for fuels kgCO2e/L_y'].fillna(0)
final_emission_df['Ef for fuels kgCO2e/L'] = final_emission_df['Ef for fuels kgCO2e/L_x'] + final_emission_df['Ef for fuels kgCO2e/L_y']
final_emission_df['VehicleSubCategory'] = (final_emission_df['VehicleSubCategory_x'].fillna('')).astype(str)
final_emission_df['FuelType'] = (final_emission_df['FuelType_x'].fillna('')).astype(str)
final_emission_df['Emission(kg/km)'] = (final_emission_df['Emission(kg/km)_x'].fillna('')).astype(str) + (final_emission_df['Emission(kg/km)_y'].fillna('')).astype(str)
final_emission_df['EEIO ef'] = (final_emission_df['EEIO ef_x'].fillna('')).astype(str) + (final_emission_df['EEIO ef_y'].fillna('')).astype(str)
final_emission_df_1 = final_emission_df[['EmployeeName','EmployeeID','WorkRegion','OrganisationName','Role','TravelledCountry','TravelledDate','WentWithGroup','NoOfEmployeesInGroup','Employees per Vehicle','TrasportMode','TotalNightsStayed','Ef for fuels kgCO2e/L','AmountSpentOnTravelModeIn$','VehicleManufacturer','VehicleSubCategory','EngineCapacity','VersionOrClass','FuelType','DistanceTravelledInTheVehicle','Fuel price per L','AmountSpentOnFuelIn$','Fuel Consumed','ElectricityConsumed','AgencyName','consumed L per 100km','Emission(kg/km)','EEIO ef']].copy()

fin_emi_df = pd.merge(final_emission_df_1, emissions_lookup_rail_air_df, on=['TrasportMode','VehicleSubCategory','VersionOrClass'] , how = 'left')
fin_emi_df['Emission(kg/km)'] = (fin_emi_df['Emission(kg/km)_x'].fillna('')).astype(str) + (fin_emi_df['Emission(kg/km)_y'].fillna('')).astype(str)
fin_emi_df['EEIO ef'] = (fin_emi_df['EEIO ef_x'].fillna('')).astype(str) + (fin_emi_df['EEIO ef_y'].fillna('')).astype(str)
fin_emi_df['Ef for fuels kgCO2e/L_x'] = fin_emi_df['Ef for fuels kgCO2e/L_x'].fillna(0)
fin_emi_df['Ef for fuels kgCO2e/L_y'] = fin_emi_df['Ef for fuels kgCO2e/L_y'].fillna(0)
fin_emi_df['Ef for fuels kgCO2e/L'] = fin_emi_df['Ef for fuels kgCO2e/L_x'] + fin_emi_df['Ef for fuels kgCO2e/L_y']
fin_emi_df_1 = fin_emi_df[['EmployeeName','EmployeeID','WorkRegion','OrganisationName','Role','TravelledCountry','TravelledDate','WentWithGroup','NoOfEmployeesInGroup','Employees per Vehicle','TrasportMode','TotalNightsStayed','Ef for fuels kgCO2e/L','AmountSpentOnTravelModeIn$','VehicleManufacturer','VehicleSubCategory','EngineCapacity','VersionOrClass','FuelType','DistanceTravelledInTheVehicle','Fuel price per L','AmountSpentOnFuelIn$','Fuel Consumed','ElectricityConsumed','AgencyName','consumed L per 100km','Emission(kg/km)','EEIO ef']].copy()
fin_emi_df_1['NoOfEmployeesInGroup'] = fin_emi_df_1['NoOfEmployeesInGroup'].fillna(1)

emp_emission_info_df1 = fin_emi_df_1
emp_emission_info_df1['AmountSpentOnTravelModeIn$'] = emp_emission_info_df1['AmountSpentOnTravelModeIn$'].fillna(0)
emp_emission_info_df1['Emission(kg/km)'] = emp_emission_info_df1['Emission(kg/km)'].replace('',0)
emp_emission_info_df1['Emission(kg/km)'] = emp_emission_info_df1['Emission(kg/km)'].astype(float)
emp_emission_info_df1['EEIO ef'] = emp_emission_info_df1['EEIO ef'].replace('',0)
emp_emission_info_df1['EEIO ef'] = emp_emission_info_df1['EEIO ef'].astype(float)

emp_emission_info_df2 = pd.merge(emp_emission_info_df1, emissions_lookup_hotel_df, on=['TrasportMode','TravelledCountry'] , how = 'left')
emp_emission_info_df2['TravelledDate'] = emp_emission_info_df2['TravelledDate_x']

emp_emission_info_df = emp_emission_info_df2[['EmployeeName', 'EmployeeID', 'WorkRegion', 'OrganisationName', 'Role',
       'TravelledCountry', 'TravelledDate', 'WentWithGroup',
       'NoOfEmployeesInGroup', 'Employees per Vehicle', 'TrasportMode',
       'TotalNightsStayed', 'Ef for fuels kgCO2e/L',
       'AmountSpentOnTravelModeIn$', 'VehicleManufacturer',
       'VehicleSubCategory', 'EngineCapacity', 'VersionOrClass', 'FuelType',
       'DistanceTravelledInTheVehicle', 'Fuel price per L',
       'AmountSpentOnFuelIn$', 'Fuel Consumed', 'ElectricityConsumed',
       'AgencyName', 'consumed L per 100km', 'Emission(kg/km)', 'EEIO ef',
       'Emissions Factor kgCO2e']]

Inter_df = pd.DataFrame()

Inter_df['EmployeeName'] = emp_emission_info_df['EmployeeName']
Inter_df['EmployeeID'] = emp_emission_info_df['EmployeeID']
Inter_df['OrganisationName'] = emp_emission_info_df['OrganisationName']
Inter_df['WorkRegion'] = emp_emission_info_df['WorkRegion']
Inter_df['TravelledMonth_Year'] = pd.to_datetime(emp_emission_info_df['TravelledDate'], format='%Y%m%d')

Inter_df['tempdate'] = Inter_df['TravelledMonth_Year'].apply(lambda x: x.strftime('%d')).astype(int)
Inter_df.loc[Inter_df['tempdate'] > 1, 'tempdate'] = (Inter_df['tempdate'].apply(lambda x: x-(x-1)))
Inter_df['tempdate'] = pd.to_datetime(Inter_df['tempdate'], format='%d')
Inter_df['TravelledMonth_Year'] = Inter_df['TravelledMonth_Year'].apply(lambda x: x.strftime('%Y/%m')) + Inter_df['tempdate'].apply(lambda x: x.strftime('/%d'))
Inter_df['TravelledCountry'] = emp_emission_info_df['TravelledCountry']
Inter_df['TrasportMode'] = emp_emission_info_df['TrasportMode']
Inter_df['Fuel Consumed'] = emp_emission_info_df['Fuel Consumed']
Inter_df['Ef for fuels kgCO2e/L'] = emp_emission_info_df['Ef for fuels kgCO2e/L']
Inter_df['consumed L/km'] = emp_emission_info_df['consumed L per 100km'] / 100
Inter_df['Fuel price per L'] = emp_emission_info_df['Fuel price per L'].astype(float)
Inter_df['AmountSpentOnFuelIn$'] = emp_emission_info_df['AmountSpentOnFuelIn$'].astype(float)
Inter_df['TotalNightsStayed'] = emp_emission_info_df['TotalNightsStayed'].fillna(0)
Inter_df['Emissions Factor kgCO2e'] = emp_emission_info_df['Emissions Factor kgCO2e'].fillna(0)
temp = pd.DataFrame()

Inter_df['Fuel Consumed'] = Inter_df['Fuel Consumed'].fillna(0)

Inter_df.loc[Inter_df['Fuel Consumed'] == 0, 'Fuel Consumed'] = (emp_emission_info_df['consumed L per 100km'] / 100).mul(emp_emission_info_df['DistanceTravelledInTheVehicle'].fillna(0))
Inter_df['Fuel Consumed'] = Inter_df['Fuel Consumed'].fillna(0)
Inter_df.loc[Inter_df['Fuel Consumed'] == 0, 'Fuel Consumed'] = Inter_df['AmountSpentOnFuelIn$'].div(Inter_df['Fuel price per L'].fillna(1))
Inter_df['Fuel Consumed'] = Inter_df['Fuel Consumed'].fillna(0)

temp['TrasportMode'] = Inter_df['TrasportMode']
ser_tm = Inter_df['TrasportMode']
temp['Fuel based emission'] = Inter_df['Fuel Consumed'].mul(Inter_df['Ef for fuels kgCO2e/L'].fillna(0))
temp['Fuel based emission'] = temp['Fuel based emission'].fillna(0)
temp.loc[temp['Fuel based emission'] == 0, 'Distance based emission'] = (emp_emission_info_df['NoOfEmployeesInGroup'].mul(emp_emission_info_df['DistanceTravelledInTheVehicle'])).mul(emp_emission_info_df['Emission(kg/km)'])
temp.loc[temp['Distance based emission'] == 0, 'spend based emission'] = emp_emission_info_df['AmountSpentOnTravelModeIn$'].fillna(0).mul(emp_emission_info_df['EEIO ef'].fillna(0))

temp['Distance based emission'] = temp['Distance based emission'].fillna(0)
temp['spend based emission'] = temp['spend based emission'].fillna(0)

temp.loc[temp['Fuel based emission'] !=0, 'CO2eCalculationMethod'] = "Fuel-based Method"
temp.loc[temp['Distance based emission'] !=0, 'CO2eCalculationMethod'] = "Distance-based Method"
temp.loc[temp['spend based emission'] !=0, 'CO2eCalculationMethod'] = "Spend-based Method"

tmp = pd.DataFrame()

tmp['EmissionCO2e'] = temp['Fuel based emission'] + temp['Distance based emission'] + temp['spend based emission']
for x in ser_tm:
    if (x == "Hotel"):
        tmp.loc[tmp['EmissionCO2e'] == 0, 'EmissionCO2e'] = Inter_df['TotalNightsStayed'].mul(Inter_df['Emissions Factor kgCO2e'])

Inter_df['EmissionCO2e'] = tmp['EmissionCO2e'].fillna(0)
Inter_df['Co2e Unit'] = "kg"
Inter_df['Category Details'] = category_name
Inter_df['FuelType'] = emp_emission_info_df['FuelType']
Inter_df['CO2eCalculationMethod'] = temp['CO2eCalculationMethod'].fillna('Hotel-Stayed Method')
Inter_df['Scope'] = scope
Inter_df['Fuel Consumed'] = emp_emission_info_df['Fuel Consumed']
Inter_df['TrasportMode'] = temp['TrasportMode']
Inter_df.to_csv('out.csv')

fin_df = Inter_df.groupby(['Category Details','EmployeeName','TrasportMode','TravelledMonth_Year','OrganisationName','WorkRegion','CO2eCalculationMethod','FuelType'], as_index=False).agg({'EmissionCO2e' :'sum'})
fin_df['Co2e Unit'] = Inter_df['Co2e Unit']
fin_df['Scope']= Inter_df['Scope']
fin_df.to_csv('final_out.csv')
print("Done")


Done
